In [1]:
import numpy as np
import pandas as pd
import os
import sqlite3
import glob
import copy
import datetime

In [2]:
def Savetosqlite3(df,DB_Name = 'F:\sqlite3\\tdx.db',sql='insert into index_daily(ts_code,trade_date,close,open,high,low,pct_chg,vol,amount,name) values(?,?,?,?,?,?,?,?,?,?,?)'):    
    conn = sqlite3.connect(DB_Name)  
    cursor = conn.cursor()    # 向sqlite3中表插入数据的SQL语句
    print('连接数据库%s成功' % (DB_Name))
    #print(df)
    refresh_date = datetime.date.today() #今天日期
    df['refresh_date'] = refresh_date
    
    for i in range(len(df)-1):
        try:
            cursor.execute(sql,df.iloc[i])    #每日的数据添加一次，不再汇总后添加
            conn.commit()             
        except Exception as e:
            print(e)        
            conn.rollback() # 回滚
            print('插入数据到表失败')               
    print (XXX has been saved in SQLite.TDX的 '+'  交易所中!' )
    conn.close()        # 关闭数据库  

In [3]:
class Getdata():
    """
    提供通达信导出的txt文本数据地址，该地址作为参数。通达信导出格式为ta间隔，日期以YYYY-MM-DD格式。
    处理后的csv格式数据保存在该目录下面
    默认地址为'D:\\tushare\\txtdata\\index'
    """    
    def __init__(self,file_addr,DB_Name,sql):
        self.file_address = file_addr
        self.sql = sql
        self.DB_Name = DB_Name

        
    def save_alldata_from_tdx(self):
        os.chdir(self.file_address)
        files_list = glob.glob('*.txt')
        print(len(files_list))
        address = os.getcwd()#返回当前目录

        for i in range(len(files_list)):
            try:
                #header =0,读入文件，txt文件中的第一行保存，然后代码和名称读出为nameN和nameM，skipfooter=1,删除最后一行的通达信数据源，delim_whitespace=True分隔符为tab
                data = pd.read_table(address+'\\'+ files_list[i],delim_whitespace=True,encoding='gb2312',skipfooter=1,parse_dates=True,infer_datetime_format=True) 
                nameN = copy.deepcopy(data.columns[0])
                nameM = copy.deepcopy(data.columns[1])
                print(nameN,nameM)
                '''不要st股票,#name1 = re.sub(r'\/' , 'x',s1)
                if ('*st' in nameM):
                    print(files_list[i] +' is *ST股票，删除中...')
                    continue;
                '''
                #header =1，再次读入文件，包含txt文本中的第一行，并把刚刚读出的代码和名称写入
                data = pd.read_table(address+'\\'+ files_list[i],delim_whitespace=True,header =1,encoding='gb2312',skipfooter=1,parse_dates=True,infer_datetime_format=True) 
                #data = pd.read_table(file_address +'\\'+ files_list[i],sep=',',encoding='gb2312',header =1,parse_dates=True,infer_datetime_format=True) 
                data.insert(0,'code',nameN)
                data.insert(8,'name',nameM)
                data.rename(index=str, columns={"开盘": "open", "收盘": "close", "成交量": "vol", "日期": "date", "最低": "low", "最高": "high", "成交额": "amount"},inplace=True)
                #data.to_csv('D'+nameN+'.csv',sep=',',encoding='gb2312')
                data['pct_chg'] = data.close.diff()/data['close'] * 100
                
                Savetosqlite3(data,self.DB_Name,self.sql)#保存到数据库
                print( nameM  +' has been saved!' )
            except UnicodeDecodeError:
                print(files_list[i] +'gb2312 codec can not decode')
            except:
                print(files_list[i] +" has Unexpected error:")      
"""                
    def getstockdata(self, code='000001', year_s=2016,month_s=1,day_s=1,year_e=2019,month_e=2,day_e=5):
        
        获取单个股票处理后的dataframe格式数据,code 默认为000001，默认start='2016-01-01',end='2019-2-5'
        
        data=pd.read_csv(self.file_address+'\\D'+ self.code+'.csv',sep=',',encoding='gb2312')
        #data.set_index('date',inplace=True)
        data.index = pd.to_datetime(data1['date'])
        data.drop(columns=['Unnamed: 0','date'],axis=1,inplace=True)
        
        return data.iloc[self.startdt:self.enddt]
"""        
                

"                \n    def getstockdata(self, code='000001', year_s=2016,month_s=1,day_s=1,year_e=2019,month_e=2,day_e=5):\n        \n        获取单个股票处理后的dataframe格式数据,code 默认为000001，默认start='2016-01-01',end='2019-2-5'\n        \n        data=pd.read_csv(self.file_address+'\\D'+ self.code+'.csv',sep=',',encoding='gb2312')\n        #data.set_index('date',inplace=True)\n        data.index = pd.to_datetime(data1['date'])\n        data.drop(columns=['Unnamed: 0','date'],axis=1,inplace=True)\n        \n        return data.iloc[self.startdt:self.enddt]\n"

In [4]:
if __name__ == "__main__":
    sql='insert into index_daily(ts_code,trade_date,open,high,low,close,vol,amount,name,pct_chg,refresh_date) values(?,?,?,?,?,?,?,?,?,?,?)'
    DB_Name = 'E:\sqlite3\\tdx.db'
    file_address = 'E:/sqlite3/'
    histry_data = Getdata(file_address,DB_Name,sql)# 生成类实体
    histry_data.save_alldata_from_tdx()  
    
    print('刘老板，任务执行完毕，请查看Navicat！')

21
399005 中小板指


D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


连接数据库E:\sqlite3\tdx.db成功
        code        date     open     high      low    close        vol  \
0     399005  2006/01/24  1457.22  1471.20  1446.44  1459.20     906549   
1     399005  2006/01/25  1456.09  1463.72  1414.45  1430.02    1156780   
2     399005  2006/02/06  1433.44  1482.33  1433.44  1482.33     812891   
3     399005  2006/02/07  1483.45  1485.69  1450.15  1463.92    1026231   
4     399005  2006/02/08  1459.67  1469.49  1446.54  1469.49     609552   
...      ...         ...      ...      ...      ...      ...        ...   
3519  399005  2020/07/17  8667.37  8887.84  8602.27  8760.51  210082521   
3520  399005  2020/07/20  8934.30  8978.35  8725.96  8972.58  221085748   
3521  399005  2020/07/21  8980.28  9049.95  8906.51  9009.21  205990003   
3522  399005  2020/07/22  9002.10  9211.55  8962.23  9103.60  221537369   
3523  399005  2020/07/23  8990.38  9113.75  8842.24  8871.88  133587670   

            amount  name   pct_chg  
0     7.880290e+08  中小板指       NaN  


D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


399300 沪深300


...

累计北上资金(亿) has been saved!
7_TOS 累计南下资金(亿)
连接数据库E:\sqlite3\tdx.db成功
      code        date      open      high       low     close  vol  amount  \
0    7_TOS  2016/12/05   4216.69   4216.69   4216.69   4216.69    0     0.0   
1    7_TOS  2016/12/06   4223.94   4223.94   4223.94   4223.94    0     0.0   
2    7_TOS  2016/12/07   4231.64   4231.64   4231.64   4231.64    0     0.0   
3    7_TOS  2016/12/08   4218.27   4218.27   4218.27   4218.27    0     0.0   
4    7_TOS  2016/12/09   4219.90   4219.90   4219.90   4219.90    0     0.0   
..     ...         ...       ...       ...       ...       ...  ...     ...   
817  7_TOS  2020/07/16  19864.11  19864.11  19864.11  19864.11    0     0.0   
818  7_TOS  2020/07/17  19895.11  19895.11  19895.11  19895.11    0     0.0   
819  7_TOS  2020/07/20  19954.99  19954.99  19954.99  19954.99    0     0.0   
820  7_TOS  2020/07/21  20016.93  20016.93  20016.93  20016.93    0     0.0   
821  7_TOS  2020/07/22  20059.69  20059.69  20059.69  20059.69  

D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


累计南下资金(亿) has been saved!
9_990011 主动股基指数
连接数据库E:\sqlite3\tdx.db成功
          code        date     open     high      low    close  vol  amount  \
0     9_990011  2011/05/16  1000.00  1000.00  1000.00  1000.00    0     0.0   
1     9_990011  2011/05/17  1001.30  1001.30  1001.30  1001.30    0     0.0   
2     9_990011  2011/05/18  1008.58  1008.58  1008.58  1008.58    0     0.0   
3     9_990011  2011/05/19  1004.79  1004.79  1004.79  1004.79    0     0.0   
4     9_990011  2011/05/20  1003.88  1003.88  1003.88  1003.88    0     0.0   
...        ...         ...      ...      ...      ...      ...  ...     ...   
2231  9_990011  2020/07/16  2125.11  2125.11  2125.11  2125.11    0     0.0   
2232  9_990011  2020/07/17  2151.66  2151.66  2151.66  2151.66    0     0.0   
2233  9_990011  2020/07/20  2183.23  2183.23  2183.23  2183.23    0     0.0   
2234  9_990011  2020/07/21  2218.10  2218.10  2218.10  2218.10    0     0.0   
2235  9_990011  2020/07/22  2236.66  2236.66  2236.66  2236.66  

D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
D:\Users\a\Anaconda3\envs\wvcpu\lib\site-packages\ipykernel_launcher.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


主动股基指数 has been saved!
刘老板，任务执行完毕，请查看Navicat！
